In [1]:
from pathlib import Path
import pandas as pd

from expense_models import load_expenses, expenses_to_dataframe

In [2]:
expenses_path = Path("2025-11-07.json")
expenses = load_expenses(expenses_path)

len(expenses)

df = expenses_to_dataframe(expenses)
df


1 months months
1.0
1 months months
1.0
1 months months
1.0
1 months months
1.0
1 months months
1.0
1 months months
1.0
1 months months
1.0
1 months months
1.0
1 months months
1.0
1 months months
1.0
1 months months
1.0
1 months months
1.0
1 months months
1.0
1 months months
1.0
3 weeks months
0.6899525657611039
3 weeks months
0.6899525657611039
1 months months
1.0
1 months months
1.0
1 months months
1.0
1 months months
1.0
1 weeks months
0.22998418858703465
1 weeks months
0.22998418858703465
1 months months
1.0
1 months months
1.0
1 months months
1.0
1 months months
1.0
1 months months
1.0
1 months months
1.0
1 months months
1.0
1 months months
1.0
28 days months
0.9199367543481386
28 days months
0.9199367543481386
30 days months
0.9856465225158627
30 days months
0.9856465225158627


,name,account_id,monthly_value_eur,monthly_value,currency,value,repeat_every,repeat_every_unit
0,Rent,activo_bank,705.000000,705.000000,EUR,705.00,1,months
1,T3,activo_bank,86.540000,86.540000,EUR,86.54,1,months
2,Contabilizei,nubank_cnpj,59.815205,369.000000,BRL,369.00,1,months
3,Energy,activo_bank,35.570000,35.570000,EUR,35.57,1,months
4,Vodafone,activo_bank,28.500000,28.500000,EUR,28.50,1,months
5,Internet,activo_bank,25.750000,25.750000,EUR,25.75,1,months
6,Heroku - Teya,nubank_cnpj,24.000000,24.000000,EUR,24.00,1,months
7,ChatGPT,activo_bank,21.261884,24.600000,USD,24.60,1,months
8,Figma,nubank_cnpj,20.000000,20.000000,EUR,20.00,1,months
9,Cursor,activo_bank,17.286085,20.000000,USD,20.00,1,months
